<a href="https://colab.research.google.com/github/redzuanabdullah/Test/blob/main/EDGAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests
from bs4 import BeautifulSoup

# Define the company CIK and filing type
cik = '0000320193'  # Example: Apple Inc.
filing_type = '10-K'
base_url = f'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type={filing_type}&count=100'

# Headers to mimic browser behavior
headers = {
    'User-Agent': 'Your Name (your-email@example.com)'
}

# Fetch filings list
response = requests.get(base_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Find report links
filing_links = []
for link in soup.find_all('a', {'id': 'documentsbutton'}):
    filing_links.append('https://www.sec.gov' + link['href'])

# Download the first 10 filings
for i, link in enumerate(filing_links[:10], start=1):
    filing_page = requests.get(link, headers=headers)
    filing_soup = BeautifulSoup(filing_page.text, 'html.parser')
    primary_doc_link = filing_soup.find('a', {'href': lambda x: x and x.endswith('.htm')})

    if primary_doc_link:
        report_url = 'https://www.sec.gov' + primary_doc_link['href']
        report_response = requests.get(report_url, headers=headers)
        filename = f'{filing_type}_report_{i}.html'
        with open(filename, 'wb') as file:
            file.write(report_response.content)
        print(f'Downloaded: {filename}')


Downloaded: 10-K_report_1.html
Downloaded: 10-K_report_2.html
Downloaded: 10-K_report_3.html
Downloaded: 10-K_report_4.html
Downloaded: 10-K_report_5.html
Downloaded: 10-K_report_6.html
Downloaded: 10-K_report_7.html
Downloaded: 10-K_report_8.html
Downloaded: 10-K_report_9.html
Downloaded: 10-K_report_10.html


In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

# Loop through all HTML files in the current directory
for file_name in os.listdir():
    if file_name.endswith('.html') and file_name.startswith('10-K'):
        with open(file_name, 'r', encoding='utf-8') as file:
            soup = BeautifulSoup(file, 'html.parser')

        # Find all tables in the HTML file
        tables = soup.find_all('table')

        excel_file = file_name.replace('.html', '.xlsx')
        with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
            if tables:
                # Extract each table and save as a sheet
                for i, table in enumerate(tables):
                    try:
                        df = pd.read_html(str(table))[0]
                        sheet_name = f"Table_{i + 1}"
                        df.to_excel(writer, sheet_name=sheet_name, index=False)
                    except ValueError as e:
                        print(f"Error parsing table {i + 1} in {file_name}: {e}")
            else:
                # Add a default sheet
                df = pd.DataFrame({"Message": [f"No tables found in {file_name}."]})
                df.to_excel(writer, sheet_name="Info", index=False)

        print(f"Saved tables to {excel_file}")
